In [30]:
import catboost as cb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [31]:
df = pd.read_csv('earthquakes.csv')

df.time = pd.to_datetime(df.time)
df = df.loc[df.time >= "1994-01-01"]
df = df.sort_values("time")
df = df.set_index("time")

df["region"] = df.place.str.split(", ", expand=True)[1]
df.region = df.region.fillna(df.place)
df.region = df.region.replace("CA", "California")
df.region = df.region.replace("B.C.", "Baja California")

regions = df.region.value_counts()
top_k = 25
top_k_regions = regions.head(top_k).index

In [32]:
model = cb.CatBoostRegressor(cat_features=['region'])
model = model.load_model('model_depth_10_2')

In [33]:
start_lag = 3
end_lag = 12
features = [
    "day",
    "dayofweek",
    "dayofyear",
    f"mag_rolling_mean_{start_lag}",
    f"mag_rolling_std_{start_lag}",
    f"depth_rolling_mean_{start_lag}",
    f"depth_rolling_std_{start_lag}",
    f"latitude_rolling_mean_{start_lag}",
    f"latitude_rolling_std_{start_lag}",
    f"longitude_rolling_mean_{start_lag}",
    f"longitude_rolling_std_{start_lag}",
    f"mag_rolling_mean_{end_lag}",
    f"mag_rolling_std_{end_lag}",
    f"depth_rolling_mean_{end_lag}",
    f"depth_rolling_std_{end_lag}",
    f"latitude_rolling_mean_{end_lag}",
    f"latitude_rolling_std_{end_lag}",
    f"longitude_rolling_mean_{end_lag}",
    f"longitude_rolling_std_{end_lag}",
] + [f"mag_lag_{i}" for i in range(start_lag, end_lag + 1)]
cat_features = ["region"]
target = "mag"

In [34]:
test = pd.read_csv(
    "https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&eventtype=earthquake&limit=20000"
)
test["region"] = test.place.str.split(", ", expand=True)[1]
test.region = test.region.fillna(test.place)
test.region = test.region.replace("CA", "California")
test.region = test.region.replace("B.C.", "Baja California")
test.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,region
0,2024-05-30T06:46:33.700Z,35.967000,-117.652833,4.1400,1.47,ml,26.0,90.0,0.019160,0.15,...,"23 km E of Little Lake, CA",earthquake,0.21,0.280000,0.149,18.0,automatic,ci,ci,California
1,2024-05-30T06:29:01.577Z,61.620300,-146.532400,29.8000,1.20,ml,NaN,NaN,NaN,0.62,...,"43 km SSE of Nelchina, Alaska",earthquake,NaN,0.200000,NaN,NaN,automatic,ak,ak,Alaska
2,2024-05-30T06:18:48.440Z,35.645667,-117.453500,6.2800,1.43,ml,26.0,61.0,0.031060,0.12,...,"14 km SSW of Searles Valley, CA",earthquake,0.17,0.300000,0.163,20.0,automatic,ci,ci,California
3,2024-05-30T06:05:53.560Z,38.832001,-122.811165,1.6600,0.75,md,10.0,164.0,0.008635,0.02,...,"8 km WNW of Cobb, CA",earthquake,0.35,0.870000,0.160,11.0,automatic,nc,nc,California
4,2024-05-30T06:05:23.996Z,28.518000,-98.593000,4.2489,1.50,ml,10.0,136.0,0.200000,0.80,...,"7 km NW of Tilden, Texas",earthquake,0.00,2.259425,0.100,8.0,automatic,tx,tx,Texas


In [35]:
live_data = pd.read_csv(
    "https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&eventtype=earthquake&limit=20000"
)
live_data.time = pd.to_datetime(live_data.time)
live_data = live_data.sort_values("time")
live_data = live_data.set_index("time")

live_data["region"] = live_data.place.str.split(", ", expand=True)[1]
live_data.region = live_data.region.fillna(live_data.place)
live_data.region = live_data.region.replace("CA", "California")
live_data.region = live_data.region.replace("B.C.", "Baja California")

live_data = live_data.loc[live_data.region.isin(top_k_regions)]

live_data.depth = live_data.depth.astype("float32")
live_data.mag = live_data.mag.astype("float32")

live_data = live_data[["depth", "mag", "region", 'latitude', 'longitude']]

live_data = live_data.groupby("region").resample("d").mean()
live_data = live_data.reset_index()
live_data.mag = live_data.mag.ffill()
live_data.depth = live_data.depth.ffill()
live_data.latitude = live_data.latitude.ffill()
live_data.longitude = live_data.longitude.ffill()

live_data["day"] = live_data.time.dt.day
live_data["month"] = live_data.time.dt.month
live_data["dayofweek"] = live_data.time.dt.dayofweek
live_data["dayofyear"] = live_data.time.dt.dayofyear

for i in range(start_lag, end_lag + 1):
    live_data[f"mag_lag_{i}"] = live_data.groupby("region").mag.shift(i)

live_data[f"mag_rolling_mean_{start_lag}"] = live_data.groupby("region").mag.transform(
    lambda x: x.rolling(window=start_lag).mean()
)
live_data[f"mag_rolling_std_{start_lag}"] = live_data.groupby("region").mag.transform(
    lambda x: x.rolling(window=start_lag).std()
)
live_data[f"depth_rolling_mean_{start_lag}"] = live_data.groupby("region").depth.transform(
    lambda x: x.rolling(window=start_lag).mean()
)
live_data[f"depth_rolling_std_{start_lag}"] = live_data.groupby("region").depth.transform(
    lambda x: x.rolling(window=start_lag).std()
)
live_data[f"latitude_rolling_mean_{start_lag}"] = live_data.groupby("region").latitude.transform(
    lambda x: x.rolling(window=start_lag).mean()
)
live_data[f"latitude_rolling_std_{start_lag}"] = live_data.groupby("region").latitude.transform(
    lambda x: x.rolling(window=start_lag).std()
)
live_data[f"longitude_rolling_mean_{start_lag}"] = live_data.groupby("region").longitude.transform(
    lambda x: x.rolling(window=start_lag).mean()
)
live_data[f"longitude_rolling_std_{start_lag}"] = live_data.groupby("region").longitude.transform(
    lambda x: x.rolling(window=start_lag).std()
)

live_data[f"mag_rolling_mean_{end_lag}"] = live_data.groupby("region").mag.transform(
    lambda x: x.rolling(window=end_lag).mean()
)
live_data[f"mag_rolling_std_{end_lag}"] = live_data.groupby("region").mag.transform(
    lambda x: x.rolling(window=end_lag).std()
)
live_data[f"depth_rolling_mean_{end_lag}"] = live_data.groupby("region").depth.transform(
    lambda x: x.rolling(window=end_lag).mean()
)
live_data[f"depth_rolling_std_{end_lag}"] = live_data.groupby("region").depth.transform(
    lambda x: x.rolling(window=end_lag).std()
)
live_data[f"latitude_rolling_mean_{end_lag}"] = live_data.groupby("region").latitude.transform(
    lambda x: x.rolling(window=end_lag).mean()
)
live_data[f"latitude_rolling_std_{end_lag}"] = live_data.groupby("region").latitude.transform(
    lambda x: x.rolling(window=end_lag).std()
)
live_data[f"longitude_rolling_mean_{end_lag}"] = live_data.groupby("region").longitude.transform(
    lambda x: x.rolling(window=end_lag).mean()
)
live_data[f"longitude_rolling_std_{end_lag}"] = live_data.groupby("region").longitude.transform(
    lambda x: x.rolling(window=end_lag).std()
)

In [36]:
live_data.head()

,region,time,depth,mag,latitude,longitude,day,month,dayofweek,dayofyear,...,longitude_rolling_mean_3,longitude_rolling_std_3,mag_rolling_mean_12,mag_rolling_std_12,depth_rolling_mean_12,depth_rolling_std_12,latitude_rolling_mean_12,latitude_rolling_std_12,longitude_rolling_mean_12,longitude_rolling_std_12
0,Alaska,2024-04-30 00:00:00+00:00,19.569136,0.806420,58.324326,-157.247962,30,4,1,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,2024-05-01 00:00:00+00:00,29.993999,0.868000,59.541299,-153.265623,1,5,2,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alaska,2024-05-02 00:00:00+00:00,29.744047,1.895727,57.191354,-156.537245,2,5,3,123,...,-155.683610,2.123977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alaska,2024-05-03 00:00:00+00:00,25.897343,1.068421,59.755204,-155.108240,3,5,4,124,...,-154.970369,1.640163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alaska,2024-05-04 00:00:00+00:00,23.463594,1.270508,59.325290,-153.977179,4,5,5,125,...,-155.207555,1.282919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
live_prediction = model.predict(live_data[features + cat_features])
print(f"Mean Absolute Error: {mean_absolute_error(live_data[target], live_prediction)}")
print(
    f"Root Mean Squared Error: {np.sqrt(mean_squared_error(live_data[target], live_prediction))}"
)
print(f"R2 Score: {r2_score(live_data[target], live_prediction)}")

In [ ]:
df_live = pd.DataFrame({'region': live_data.region, 'mag': live_data[target], 'prediction': live_prediction, 'time': live_data.time})
df_live = df_live.sort_values(by='time')

k = 5
for region in df_live.region.unique():
    df_region = df_live.loc[df_live.region == region]
    df_region.plot(title=f"Forecast for region: {region}", x='time', y=['prediction', 'mag'])
    k -= 1
    if not k:
        break